# 04 - Avaliação e Resultados

Este notebook realiza a avaliação dos modelos de regressão utilizados na previsão de preços de smartphones e apresenta as conclusões finais do projeto. Aqui os modelos são treinados novamente e avaliados com base nas métricas de desempenho.

In [ ]:

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import matplotlib.pyplot as plt


In [ ]:

# Carregar o dataset limpo gerado na etapa de ETL
df = pd.read_csv("/content/dados_limpos_para_analise.csv")
df.head()


In [ ]:

# Separar variáveis independentes (X) e variável alvo (y)
X = df.drop(columns=["price_usd"])
y = df["price_usd"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train.shape, X_test.shape


In [ ]:

# Treinamento dos modelos

# Regressão Linear
modelo_lr = LinearRegression()
modelo_lr.fit(X_train, y_train)

# Random Forest Regressor
modelo_rf = RandomForestRegressor(
    n_estimators=300,
    random_state=42,
    n_jobs=-1
)
modelo_rf.fit(X_train, y_train)


In [ ]:

# Fazer previsões com os dois modelos
y_pred_lr = modelo_lr.predict(X_test)
y_pred_rf = modelo_rf.predict(X_test)


In [ ]:

# Função auxiliar para calcular métricas
def avaliar(nome, y_real, y_pred):
    rmse = mean_squared_error(y_real, y_pred, squared=False)
    mae = mean_absolute_error(y_real, y_pred)
    r2 = r2_score(y_real, y_pred)

    return {
        "Modelo": nome,
        "RMSE": rmse,
        "MAE": mae,
        "R²": r2
    }

resultados = [
    avaliar("Regressão Linear", y_test, y_pred_lr),
    avaliar("Random Forest", y_test, y_pred_rf)
]

df_resultados = pd.DataFrame(resultados)
df_resultados


## Interpretação das Métricas

Os resultados obtidos mostram que o modelo Random Forest apresenta desempenho superior ao da Regressão Linear, com valores menores de RMSE e MAE, além de um R² mais alto. Isso indica que o Random Forest consegue capturar melhor as relações entre as variáveis técnicas e o preço dos smartphones. A Regressão Linear, apesar de simples e interpretável, mostra limitações para representar a complexidade do conjunto de dados.

In [ ]:

# Gráfico: Preço real vs preço previsto pelo Random Forest
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred_rf, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color="red")
plt.xlabel("Preço Real (price_usd)")
plt.ylabel("Preço Previsto (Random Forest)")
plt.title("Comparação entre Preço Real e Preço Previsto")
plt.show()


In [ ]:

# Importância das variáveis no modelo Random Forest
importances = modelo_rf.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10,6))
plt.bar(range(10), importances[indices][:10])
plt.xticks(range(10), X.columns[indices][:10], rotation=45, ha="right")
plt.title("Importância das Variáveis (Top 10)")
plt.tight_layout()
plt.show()


## Análise da Importância das Variáveis

A análise das importâncias indica quais atributos mais contribuem para a previsão do preço dos smartphones no modelo Random Forest. Em geral, variáveis como memória RAM, armazenamento interno, qualidade das câmeras e suporte ao 5G tendem a aparecer entre as mais relevantes, o que está alinhado ao comportamento real do mercado.

# Conclusão

Com base nas métricas calculadas e nas análises realizadas, o Random Forest Regressor foi selecionado como o melhor modelo para prever o preço de smartphones no conjunto de dados analisado. Ele apresentou maior precisão, melhor capacidade de generalização e comportamento coerente com os padrões identificados na análise exploratória. Assim, o modelo final é capaz de estimar o preço de um dispositivo a partir de suas especificações técnicas, oferecendo suporte para estudos de mercado e análises comparativas no contexto da indústria de smartphones.